# ReAct & RAG

In [ ]:
%pip install langchain openai
%pip install openai
%pip install dalle-pytorch
%pip install dalle-mini
%pip install torch torchvision
%pip install transformers
%pip install sentence_transformers  

In [ ]:
from dotenv import load_dotenv
from langchain.tools import Tool
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatOpenAI
import openai
import os

# Load environment variables from .env file
load_dotenv()
api_key = os.getenv('OPEN_AI_API_KEY_CAR')
print(api_key)

openai.api_key = api_key

In [8]:
# Define tools
def calculator(input):
    return eval(input)

def weather(input):
    return "The weather is sunny."

tools = [Tool(name="Calculator", func=calculator, description="Performs calculations.")]
weather_tools = [Tool(name="Weather", func=weather, description="Returns the weather.")]

# Initialize the agent
llm = ChatOpenAI(temperature=0, model="gpt-4", openai_api_key=api_key)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
weather_agent = initialize_agent(weather_tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)    

# Test the agent
result = agent.run("What is 2+2?")
result2 = weather_agent.run("What is the weather?")

print(result)
print(result2)



> Entering new AgentExecutor chain...
I need to use the Weather tool to find out the current weather.
Action: Weather
Action Input: current location
Observation: The weather is sunny.
Thought:I now know the final answer.
Final Answer: The weather is sunny.

> Finished chain.
The weather is sunny.


In [ ]:
from dalle_pytorch import DALLE
import torch
from torchvision import transforms
from PIL import Image

# Load pre-trained model (example setup)
model = DALLE.load_model("path/to/your/model.pt", map_location="cpu")

# Define the text prompt
prompt = "A sleek, red sports car with a spoiler"

# Generate the image
images = model.generate_images(prompt, num_images=1)

# Save the image
images[0].save("car_image.png")


In [ ]:
import openai

# Load the CarGAN model
model = openai.Model("cargan")

# Define the text prompt
prompt = "A sleek, red sports car with a spoiler"

# Generate the image
response = model.generate_image(prompt)

# Save the image to a file
image = response["image"]
with open("car_image.png", "wb") as f:
    f.write(image)

# RaG


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Sample data
documents = [
    "The Eiffel Tower is located in Paris.",
    "AI is transforming industries like healthcare and finance.",
    "RAG systems enhance AI by combining retrieval with generation."
]

# Create vector database
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(documents, embeddings)

# Build QA pipeline
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(temperature=0, model="gpt-4")
qa = RetrievalQA.from_chain_type(llm, retriever=retriever)

# Test the system
question = "Where is the Eiffel Tower located?"
answer = qa.run(question)
print(answer)


# ReAct project

```
travel_planner/
│
├── app.py            # Main application file
├── weather.py        # Module to handle weather-related functions
├── currency.py       # Module to handle currency conversion
├── templates/        # For HTML templates (if using Flask)
│   └── index.html
└── static/           # For CSS and JS files (if needed for Flask UI)
```

# RAG project

```
research_assistant/
│
├── app.py              # Main application logic
├── document_processor.py  # Module for document ingestion
├── faiss_handler.py    # Module to handle FAISS operations
├── embeddings_utils.py # Module for embedding generation
└── data/               # Folder to store documents


```

In [ ]:
%pip install openai faiss-cpu numpy pandas
%pip install pdfplumber  # For PDF parsing


In [ ]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
        return text

def process_document(file_path):
    if file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    else:
        with open(file_path, "r") as file:
            return file.read()


In [ ]:
import openai
import numpy as np

openai.api_key = "your_openai_api_key"

def generate_embeddings(texts):
    embeddings = []
    for text in texts:
        response = openai.Embedding.create(
            input=text,
            model="text-embedding-ada-002"
        )
        embeddings.append(response["data"][0]["embedding"])
    return np.array(embeddings)


In [ ]:
import faiss
import numpy as np

class FAISSHandler:
    def __init__(self, embedding_dim):
        self.index = faiss.IndexFlatL2(embedding_dim)

    def add_embeddings(self, embeddings):
        self.index.add(embeddings)

    def search(self, query_embedding, top_k=5):
        distances, indices = self.index.search(query_embedding, top_k)
        return distances, indices


In [ ]:
from document_processor import process_document
from embeddings_utils import generate_embeddings
from faiss_handler import FAISSHandler
import numpy as np

class ResearchAssistant:
    def __init__(self, embedding_dim):
        self.faiss_handler = FAISSHandler(embedding_dim)
        self.texts = []

    def add_document(self, file_path):
        text = process_document(file_path)
        self.texts.append(text)
        return text

    def build_index(self):
        embeddings = generate_embeddings(self.texts)
        self.faiss_handler.add_embeddings(embeddings)

    def answer_query(self, query, top_k=5):
        query_embedding = generate_embeddings([query])
        distances, indices = self.faiss_handler.search(np.array(query_embedding), top_k)
        results = [self.texts[idx] for idx in indices[0]]
        return results

if __name__ == "__main__":
    assistant = ResearchAssistant(embedding_dim=1536)  # Dimension for text-embedding-ada-002

    # Add documents
    assistant.add_document("data/sample1.pdf")
    assistant.add_document("data/sample2.txt")

    # Build the index
    assistant.build_index()

    # Query the assistant
    query = "What are the main points in the document?"
    results = assistant.answer_query(query)
    
    print("Results:")
    for res in results:
        print(res)

